In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torchvision.transforms import InterpolationMode
from torchvision import models, datasets, transforms
from torch.autograd.functional import jacobian
import math
from utils import *

/home/deponce/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:115.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
def C(i):
    if i == 0:
        return 1/(2*math.sqrt(2))
    else:
        return 1/2
def getDCTCoef(size = (8,8)):
    DCT_coef = torch.zeros(size).double()
    for i in range(size[0]):
        for k in range(size[1]):
            C_i = C(i)
            DCT_coef[i,k]=C_i*math.cos((2*k+1)*i*math.pi/16)
    return DCT_coef
DCT_coef = getDCTCoef((8,8))

In [3]:
IDCT_coef = torch.inverse(DCT_coef)

In [4]:
resize8 = transforms.Resize((8, 8))
def process(blocks):
    return resize8(blocks[None,None,:])

In [5]:
blocks = torch.ones([17,17]).double()
t1 = jacobian(process, blocks)[0,0]
t1.shape

torch.Size([8, 8, 17, 17])

In [6]:
torch.sum(torch.abs(t1))

tensor(64., dtype=torch.float64)

In [7]:
t1[:,:,1,1]

tensor([[0.3164, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],
       dtype=torch.float64)

In [8]:
t1[:,:,3,2]

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.2148, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],
       dtype=torch.float64)

In [246]:
blocks = torch.ones([8,8]).double()
def IDCT(blocks):
    return IDCT_coef@(blocks[None,None,:])@IDCT_coef.transpose(1,0)

In [247]:
def DCT(blocks):
    return DCT_coef@(blocks[None,None,:])@DCT_coef.transpose(1,0)

In [248]:
radblocks = torch.tensor(np.random.rand(8,8)).double()

In [253]:
IDCTjacobian = jacobian(IDCT, radblocks)[0,0]

In [250]:
DCTjacobian = jacobian(DCT, blocks)[0,0]

In [251]:
DCTjacobian.shape

torch.Size([8, 8, 8, 8])

In [245]:
DCTjacobian.shape

torch.Size([8, 8, 8, 8])

In [23]:
DCTjacobian[0,0].shape

torch.Size([8, 8])

In [24]:
resize224 = transforms.Resize((224,224))

In [25]:
import torch.autograd
x = torch.tensor(np.random.rand(1,1,500,500))
x.requires_grad = True
y = resize224(x)
y.backward(gradient=torch.tensor(np.random.rand(1,1,224,224))/2)
x.grad.shape

torch.Size([1, 1, 500, 500])

In [27]:
img = torch.ones(1,14,14)

In [28]:
img.shape

torch.Size([1, 14, 14])

imgAddAxis = img[None,:]
img_shape = img.shape[-2:]
DCTBlock = block_dct(blockify(imgAddAxis, 8))
DCTBlock.requires_grad = True
RecoverImg = deblockify(block_idct(DCTBlock), (img_shape[0], img_shape[1]))
RecoverSizeImg = resize224(RecoverImg)
RecoverSizeImg.backward(gradient=torch.tensor(np.zeros([1,8,8,8])))
DCTBlock.grad

In [30]:
radblocks = torch.tensor(np.random.rand(14,14)).double()

In [31]:
resize8 = transforms.Resize((8, 8))
def process(DCTblocks):
    blockSize = DCTblocks.shape
    DCT_coef = getDCTCoef(blockSize)
    IDCT_coef = torch.inverse(DCT_coef)
    blocks = IDCT_coef@DCTblocks@(IDCT_coef.transpose(1,0))
    resizeBlock = resize8(blocks[None,None,:])
    DCT_coef8 = getDCTCoef((8,8))
    return DCT_coef8@resizeBlock[0,0]@(DCT_coef8.transpose(1,0))

In [ ]:
def process(DCTblocks):
    blockSize = DCTblocks.shape
    DCT_coef = getDCTCoef(blockSize)
    IDCT_coef = torch.inverse(DCT_coef)
    blocks = IDCT_coef@DCTblocks@(IDCT_coef.transpose(1,0))
    resizeBlock = resize8(blocks[None,None,:])
    DCT_coef8 = getDCTCoef((8,8))
    return DCT_coef8@resizeBlock[0,0]@(DCT_coef8.transpose(1,0))

In [307]:
def resizefn(DCTblocks): 
    return transforms.Resize((24, 24))(DCTblocks[None,None,:])